## Baichuan 推理

### 配置实验环境

In [1]:
from _common import *
from transformers import TextStreamer
device_ids = [0, 1]
select_device(device_ids)

[2023-07-02 22:28:00,199] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2023-07-02 22:28:00,675 - modelscope - INFO - PyTorch version 2.0.1 Found.
2023-07-02 22:28:00,676 - modelscope - INFO - Loading ast index from /home/hackathon/.cache/modelscope/ast_indexer
2023-07-02 22:28:00,700 - modelscope - INFO - Loading done! Current index file version is 1.6.2, with md5 ddf811ee982377c1357284a2bfda3dec and a total number of 861 components indexed
2023-07-02 22:28:01,367 - modelscope - INFO - [0, 1]
2023-07-02 22:28:01,512 - modelscope - INFO - Using device: cuda:0,1


device(type='cuda', index=0)

### 导入Model, Tokenizer
Note: 你需要设置CKPT_FPATH的内容, 指向`.bin`文件, 或`.pth`文件

In [2]:
CKPT_FAPTH = '/home/hackathon/my_git/agent/runs/baichuan/v10-20230702-172449/output_best/pytorch_model.bin'
LORA_TARGET_MODULES = ['W_pack']

model_dir = snapshot_download('baichuan-inc/baichuan-7B', 'v1.0.5')
model, tokenizer = get_baichuan7B_model_tokenizer(model_dir)
model.bfloat16()  # Consistent with training

2023-07-02 22:28:03,375 - modelscope - INFO - Model revision not specified, use default: master in development mode
2023-07-02 22:28:03,375 - modelscope - INFO - Development mode use revision: master
2023-07-02 22:28:03,695 - modelscope - INFO - model_config: BaiChuanConfig {
  "architectures": [
    "BaiChuanForCausalLM"
  ],
  "auto_map": {
    "AutoConfig": "configuration_baichuan.BaiChuanConfig",
    "AutoModelForCausalLM": "modeling_baichuan.BaiChuanForCausalLM"
  },
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "baichuan",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "pad_token_id": 0,
  "rms_norm_eps": 1e-06,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.30.2",
  "use_cache": true,
  "vocab_size": 64000
}

The model weights are not tied. Please use the `tie_weights` method 

BaiChuanForCausalLM(
  (model): Model(
    (embed_tokens): Embedding(64000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x DecoderLayer(
        (self_attn): Attention(
          (W_pack): Linear(in_features=4096, out_features=12288, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): RotaryEmbedding()
        )
        (mlp): MLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): RMSNorm()
        (post_attention_layernorm): RMSNorm()
      )
    )
    (norm): RMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=64000, bias=False)
)

### 导入Lora

In [3]:
LORA_RANK = 8
LORA_ALPHA = 32
LORA_DROPOUT_P = 0  # Arbitrary value
lora_config = LoRAConfig(
    replace_modules=LORA_TARGET_MODULES,
    rank=LORA_RANK,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT_P,
    pretrained_weights=CKPT_FAPTH)
logger.info(f'lora_config: {lora_config}')
Swift.prepare_model(model, lora_config)

2023-07-02 22:28:14,108 - modelscope - INFO - lora_config: LoRAConfig(rank=8, replace_modules=['W_pack'], lora_alpha=32, lora_dropout=0, merge_weights=True, use_merged_linear=False, enable_lora=None, fan_in_fan_out=False, bias='none', only_lora_trainable=True, pretrained_weights='/home/hackathon/my_git/agent/runs/baichuan/v10-20230702-172449/output_best/pytorch_model.bin')


BaiChuanForCausalLM(
  (model): Model(
    (embed_tokens): Embedding(64000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x DecoderLayer(
        (self_attn): Attention(
          (W_pack): Linear(in_features=4096, out_features=12288, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): RotaryEmbedding()
        )
        (mlp): MLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): RMSNorm()
        (post_attention_layernorm): RMSNorm()
      )
    )
    (norm): RMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=64000, bias=False)
)

### 导入Dataset

In [4]:
test_dataset = make_dataset('validation', lambda system, user, assistant:
                            {'system': system, 'user': user, 'assistant': assistant})

2023-07-02 22:28:28,832 - modelscope - INFO - No subset_name specified, defaulting to the default
2023-07-02 22:28:29,317 - modelscope - WARNING - Reusing dataset ms_hackathon_23_agent_train_dev (/home/hackathon/.cache/modelscope/hub/datasets/modelscope/ms_hackathon_23_agent_train_dev/master/data_files)
2023-07-02 22:28:29,318 - modelscope - INFO - Generating dataset ms_hackathon_23_agent_train_dev (/home/hackathon/.cache/modelscope/hub/datasets/modelscope/ms_hackathon_23_agent_train_dev/master/data_files)
2023-07-02 22:28:29,318 - modelscope - INFO - Reusing cached meta-data file: /home/hackathon/.cache/modelscope/hub/datasets/modelscope/ms_hackathon_23_agent_train_dev/master/data_files/941b733ec0354c2172a3386d8788bb37


Extracting data files: 0it [00:00, ?it/s]

100%|██████████| 285/285 [00:00<00:00, 1566679.74it/s]


### 推理

In [5]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
for d in test_dataset[:5]:
    system = d['system']
    user = d['user']
    assistant = d['assistant']
    input_ids = tokenize_function(system, user, None, tokenizer)['input_ids']
    print(f'[TEST]{tokenizer.decode(input_ids)}', end='')
    input_ids = torch.tensor(input_ids)[None].cuda()
    attention_mask = torch.ones_like(input_ids)
    generate_ids = model.generate(input_ids=input_ids, max_new_tokens=512,
                                  attention_mask=attention_mask,
                                  streamer=streamer, pad_token_id=tokenizer.eos_token_id, 
                                  temperature=0.7, top_k=50, top_p=0.7, do_sample=True)
    print()
    print(f'[LABELS]{assistant}')
    print('-----------------------------------------------------------------------------------')
    # input('next[ENTER]')

[TEST] 你是达摩院的ModelScopeGPT(魔搭助手)，你是个大语言模型， 是2023年达摩院的工程师训练得到的。你有多种能力，可以通过插件集成魔搭社区的模型api来回复用户的问题，还能解答用户使用模型遇到的问题和模型知识相关问答。1. {"plugin_name": "modelscope_speech-generation", "plugin_owner": "ModelScopeGPT", "plugin_type": "default", "plugin_schema_for_model": {"name": "modelscope_speech-generation", "description": "针对回复的内容，用语音表示，同时可以选择是男声或者女声", "url": "http://90.49.118.175:2603/", "paths": [{"name": "modelscope_speech-generation", "model_id": "/damo/speech_sambert-hifigan_tts_zh-cn_16k", "method": "post", "description": "针对回复的内容，用语音表示，同时可以选择是男声或者女声", "parameters": [{"name": "text", "description": "要转成语音的文本", "required": "True"}, {"name": "gender", "description": "用户身份", "required": "True"}]}]}}

2. {"plugin_name": "modelscope_speech-generation", "plugin_owner": "ModelScopeGPT", "plugin_type": "default", "plugin_schema_for_model": {"name": "modelscope_speech-generation", "description": "针对回复的内容，用语音表示，同时可以选择是男声或者女声", "url": "http://132.94.116.115:5983/", "paths": [{"name": "modelscope_spee